In [66]:
""" Importando bibliotecas """
import os
import csv
import json

basePath = 'B:/timetabling-UENF/Files/data/Raw Data/Salas/Extracting Data/'
rootPath = basePath + 'CSV/raw'
savePath = basePath + 'CSV/cleaned'

In [13]:
""" Get all file paths in a directory"""

folders = []

def get_filepaths(directory):
    file_paths = []  # List which will store all of the full filepaths.
    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for directory in directories:
            folders.append(directory)
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.
    return file_paths  # Self-explanatory.
  
# Run the above function and store its results in a variable.
fullFilePaths = get_filepaths(rootPath)
for item in fullFilePaths:
    print(item)

B:\timetabling-UENF\Files\data\Raw Data\Salas\Extracting Data\CSV\raw\2022.2\salas 2022-2 v12_08_2022-v25-11-00.csv
B:\timetabling-UENF\Files\data\Raw Data\Salas\Extracting Data\CSV\raw\2022.2\salas 2022-2 v12_08_2022-v25-11-01.csv
B:\timetabling-UENF\Files\data\Raw Data\Salas\Extracting Data\CSV\raw\2022.2\salas 2022-2 v12_08_2022-v25-11-02.csv
B:\timetabling-UENF\Files\data\Raw Data\Salas\Extracting Data\CSV\raw\2022.2\salas 2022-2 v12_08_2022-v25-11-03.csv
B:\timetabling-UENF\Files\data\Raw Data\Salas\Extracting Data\CSV\raw\2022.2\salas 2022-2 v12_08_2022-v25-11-04.csv
B:\timetabling-UENF\Files\data\Raw Data\Salas\Extracting Data\CSV\raw\2022.2\salas 2022-2 v12_08_2022-v25-11-05.csv
B:\timetabling-UENF\Files\data\Raw Data\Salas\Extracting Data\CSV\raw\2022.2\salas 2022-2 v12_08_2022-v25-11-06.csv
B:\timetabling-UENF\Files\data\Raw Data\Salas\Extracting Data\CSV\raw\2022.2\salas 2022-2 v12_08_2022-v25-11-07.csv
B:\timetabling-UENF\Files\data\Raw Data\Salas\Extracting Data\CSV\raw\20

In [23]:
""" Saving CSV as utf-8 for Excel and as latin-1 for VSCode """

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def actuallySaveCSV(path, encoding, csvData):
    with open(path, 'w', newline='', encoding=encoding) as file: 
        csv_writer = csv.writer(file)
        csv_writer.writerows(csvData)

def saveCSV(folderName, fileName, csvData, folderDistinction, encoding):
    directory = os.path.join(savePath, folderDistinction + '-' + encoding, folderName)
    create_directory(directory)

    file_path = os.path.join(directory, fileName)
    actuallySaveCSV(file_path, encoding, csvData)

def saveCSVs(folderName, fileName, csvData):
    saveCSV(folderName, fileName, csvData, "VSCode", 'utf-8')
    saveCSV(folderName, fileName, csvData, "Excel", 'latin-1')

def saveCompleteCSVs(CSVs):
    for key in CSVs:
        if key != "clean" and len(CSVs[key]) > 0:
            saveCSVs(key, key+".csv", CSVs[key])


In [24]:
""" CLEAN CSVs:
    - Open CSV
    - Remove all \n from the cells
    - Save the new CSV
"""
localFilePath = fullFilePaths

def cleanCSVs():
  # local_file_path = [fullFilePaths[0]]
  CSVs = {
    "full": [],
    "clean": [],
    "2022.1": [],
    "2022.2": [],
    "2023.1": [],
    "2023.2": [],
    "2024.1": [],
    "else": []
  }

  for filePath in localFilePath:
    CSVs["clean"] = []
    folderFileName = filePath.split('raw\\')[-1]
    splittedFolderName = folderFileName.split('\\')
    folderName = splittedFolderName[0]
    fileName = splittedFolderName[-1]
    
    with open(filePath, 'r', encoding='utf-8') as file:
      csv_reader = csv.reader(file)
      for row in csv_reader:
        cleanRow = []
        for col in row:
          clean_col = col.replace("\n", " ")
          cleanRow.append(clean_col)
        CSVs["clean"].append(cleanRow)
        CSVs["full"].append(cleanRow)
    # print(splittedFolderName)
    if '2022.1' in splittedFolderName:
      appendTo = "2022.1"
    elif '2022.2' in splittedFolderName:
      appendTo = "2022.2"
    elif '2023.1' in splittedFolderName:
      appendTo = "2023.1"
    elif '2023.2' in splittedFolderName:
      appendTo = "2023.2"
    elif '2024.1' in splittedFolderName:
      appendTo = "2024.1"
    elif '2024.2' in splittedFolderName:
      appendTo = "2024.2"
    else:
      appendTo = "else"
    CSVs[appendTo].append(CSVs["clean"])
    saveCSVs(folderName, fileName, CSVs["clean"])
  saveCompleteCSVs(CSVs)

cleanCSVs()

In [65]:
""" GET all Subjects and professores from full CSV """


fullCSVFilePath = basePath + "cleaned/VSCode-utf-8/full/full.csv"
profs = ['Profa', 'profa', 'profs', 'Prof', 'prof']
exclusions = [ "Semestre", "Capacidade", "Horário", "Legendas", "SEGUNDA", "TERÇA", "QUARTA", "QUINTA", "SEXTA", "SÁBADO",
    "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00",
]

def splitProfAndSubject(col):
    splitted = []
    for prof in profs:
        if prof in col:
            splittedData = col.split(prof)
            reunite = []
            for item in splittedData:
                cleaned = item.replace(".", "").strip()
                reunite.append(cleaned)
            splitted = reunite
            break
    if len(splitted) > 2:
        splitted = [splitted[0] + " prof" + splitted[1], splitted[-1]]
        # print(splitted)
    # print(len(splitted))
    return splitted

def processCSVData(csv_reader):
    data = { "prof": [], "subj": [], "else": [] }
    for row in csv_reader:
        for col in row:
            size = len(col)
            if size > 0 and not any(exclusion in col for exclusion in exclusions):
                condition = any(prof in col for prof in profs)
                if condition:
                    splitted = splitProfAndSubject(col)
                    data["subj"].append(splitted[0])
                    data["prof"].append(splitted[-1])
                else:
                    data["else"].append(col)
    data["prof"] = sorted(list(set(data["prof"])))
    data["subj"] = sorted(list(set(data["subj"])))
    data["else"] = sorted(list(set(data["else"])))
    return data
    # for item in data["else"]:
    #     print(item)

def getSubjectsAndProfessors(fullCSVFilePath):
    with open(fullCSVFilePath, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        data = processCSVData(csv_reader)
        return data

subjectProfessorData = getSubjectsAndProfessors(fullCSVFilePath)

print(subjectProfessorData)

{'prof': ['', 'A Policani', 'A Stumbo', 'Afonso', 'Afonso Rangel', 'Alcimar', 'Aldo', 'Alessandra', 'Alexandre', 'Alexandre EC', 'Alexandre Stumbo', 'Alzimar', 'Ana', 'Ana Senna', 'André', 'André Guimarães', 'Angelus', 'Annabell', 'Arica', 'Ausberto', 'Bianka', 'Bolsista', 'Bruna', 'C Maurício', 'Carlos', 'Carlos E', 'Carlos Eduardo', 'Carlos Leonardo', 'Carlos M', 'Carlos Maurício', 'Cibele', 'Crisna', 'Cristiane', 'Cristiane Aguiar', 'Cristina', 'DIVERSOS', 'Dayvison', 'Delson', 'Denise', 'Denise Turma C', 'Diego', 'Diversos', 'Djalma', 'Dylmar', 'Décio', 'E Atem', 'Edmilson', 'Edson', 'Eduardo', 'Eduardo Atem', 'Elaine', 'Elba', 'Ellen', 'Everton', 'F Luna', 'Felipe', 'Fernando', 'Fernando Luna', 'Frederico', 'Gines', 'Glória', 'Gudelia', 'Guilherme', 'Gustavo', 'Gustavo Xavier', 'Herval', 'Herval Paes', 'Ivo', 'Jacqueline', 'Jan', 'Jefferson', 'Jonas', 'Jorge Hernandez', 'José Arica', 'José Augusto', 'José Nilson', 'João', 'Juraci', 'L Guillermo', 'L Mariano', 'L Rivera', 'L Rivera

In [68]:
""" Save Subjects and Professors as JSON """
saveJSONPath = basePath + "json/CCTRoomsSubjectsAndProfessors.json"
def saveSubjectsAndProfessorsAsJSON(data):
    with open(saveJSONPath, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

saveSubjectsAndProfessorsAsJSON(subjectProfessorData)